<a href="https://colab.research.google.com/github/LuanDavid7/AI_EKT/blob/main/AI_Answer_EKT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**PRIMEIRO CÓDIGO**

In [ ]:
import requests

# Configurações da API
API_KEY = "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"  # Sua chave do Groq
MODEL = "llama3-70b-8192"  # Modelo Llama 3 70B (ou "llama3-8b-8192" para versão leve)
URL = "https://api.groq.com/openai/v1/chat/completions"

def chat_llama3():
    print("\n🔵 Bem-vindo ao Chat Llama 3! (Digite 'sair' para encerrar)")
    historico = []

    while True:
        # Pega a pergunta do usuário
        pergunta = input("\nVocê: ")

        if pergunta.lower() in ["sair", "exit"]:
            print("🟡 Encerrando chat...")
            break

        # Adiciona ao histórico (opcional, para contexto)
        historico.append({"role": "user", "content": pergunta})

        # Envia para a API
        resposta = requests.post(
            URL,
            headers={"Authorization": f"Bearer {API_KEY}"},
            json={
                "model": MODEL,
                "messages": historico,
                "temperature": 0.7
            }
        )

        # Processa a resposta
        if resposta.status_code == 200:
            resposta_ia = resposta.json()["choices"][0]["message"]["content"]
            print(f"\nLlama 3: {resposta_ia}")
            historico.append({"role": "assistant", "content": resposta_ia})  # Atualiza histórico
        else:
            print(f"🔴 Erro: {resposta.text}")

# Inicia o chat
if __name__ == "__main__":
    chat_llama3()


🔵 Bem-vindo ao Chat Llama 3! (Digite 'sair' para encerrar)


KeyboardInterrupt: Interrupted by user

In [ ]:
import requests

# Configurações da API
API_KEY = "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"  # Sua chave do Groq
MODEL = "llama3-70b-8192"  # Modelo Llama 3 70B (ou "llama3-8b-8192" para versão leve)
URL = "https://api.groq.com/openai/v1/chat/completions"

def chat_llama3():
    print("\n🔵 Bem-vindo à Neoenergia Elektro! Digite 'sair' para encerrar.")
    historico = []

    # Define o papel inicial da IA
    historico.append({"role": "system", "content": """
Você é um atendente da Neoenergia Elektro, uma distribuidora de energia.
Seu objetivo é ajudar os clientes a resolverem seus problemas de energia com empatia e eficiência.
Busque compreender o problema real do cliente, mesmo que a descrição seja confusa.
Pergunte perguntas esclarecedoras para entender a situação e oferecer a melhor solução.
Apresente-se como um atendente da Elektro e identifique a si mesmo como o atendente responsável pelo chamado.
Os problemas mais comuns são: falta de energia (geral, específica, falta de fase, sigfi), risco de morte (fio partido, poste abalroado), níveis de tensão (baixa, oscilação), e defeitos (furto, falha em ramal, medidor, objeto estranho na rede).
Você deve manter um tom profissional, amável, e eficiente.
"""})

    while True:
        pergunta = input("\nVocê: ")

        if pergunta.lower() in ["sair", "exit"]:
            print("🟡 Encerrando atendimento...")
            break

        historico.append({"role": "user", "content": pergunta})

        resposta = requests.post(
            URL,
            headers={"Authorization": f"Bearer {API_KEY}"},
            json={
                "model": MODEL,
                "messages": historico,
                "temperature": 0.7
            }
        )

        if resposta.status_code == 200:
            resposta_ia = resposta.json()["choices"][0]["message"]["content"]
            print(f"\nAtendente Elektro: {resposta_ia}")
            historico.append({"role": "assistant", "content": resposta_ia})
        else:
            print(f"🔴 Erro: {resposta.text}")

if __name__ == "__main__":
    chat_llama3()


In [ ]:
import requests
import json
import re
import time

# API Configuration
API_KEY = "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"
MODEL = "llama3-70b-8192"  # Using Llama 3 70B (can change to "llama3-8b-8192" for lighter version)
URL = "https://api.groq.com/openai/v1/chat/completions"

# Common issue types and subtypes for classification
ISSUE_TYPES = {
    "FALTA DE ENERGIA": ["FALTA DE ENERGIA GERAL", "FALTA DE ENERGIA", "FALTA DE FASE",
                         "SIGFI FALTA DE ENERGIA FOTOVOLTAICO", "FALTA DE FASE GERAL",
                         "Falta de Energia em Medidor Eletronico", "Falta de Fase em Medidor Eletrônico"],

    "RISCO DE MORTE": ["OUTROS", "FIO PARTIDO NA REDE PRIMARIA", "FIO PARTIDO NO RAMAL DE SERVICO",
                       "FIO PARTIDO NA REDE SECUNDARIA", "POSTE ABALROADO SEM FIO PARTIDO",
                       "POSTE ABALROADO COM FIO PARTIDO"],

    "PROVAVEL DEFEITO FALHA": ["FURTO FIACÃO/PADRÃO", "FALHA EM RAMAL DE SERVICO",
                              "FIO DANDO FOGO NO RAMAL DE SERVICO", "FALHA EM MEDIDOR",
                              "OBJETO ESTRANHO NA REDE SECUNDARIA", "OBJETO ESTRANHO NA REDE PRIMARIA",
                              "POSTE PADR?O QUEBRADO", "FIO DANDO FOGO NA REDE PRIMARIA",
                              "FIO DANDO FOGO COM ARVORE NA REDE", "FURTO EQUIPAMENTO",
                              "ELO FUSIVEL QUEIMADO/CAIDO", "FIO DANDO FOGO NA REDE SECUNDARIA",
                              "CHOQUE EM INSTALAC?O", "FALHA EM QUICK-LAG", "FURTO REDE",
                              "INVERSÃO DE FASE"],

    "NIVEIS DE TENSÃO": ["TENSÃO BAIXA", "OSCILAC?O DE TENSÃO", "TENSÃO ALTA"],

    "OUTROS": ["TRANSFORMADOR"],

    "EMERGENCIA MEDICA": ["HOSPITAL COM PROBLEMA", "EQUIPAMENTO SUPORTE A VIDA"]
}

# Investigation questions to determine if it's an internal problem
INVESTIGATION_QUESTIONS = {
    "FALTA DE ENERGIA": [
        "Você poderia me informar se a falta de energia afeta apenas sua residência ou se os vizinhos também estão sem energia?",
        "Você já verificou o quadro de disjuntores interno da sua residência para ver se algum está desarmado?",
        "Verificou também o disjuntor geral no padrão de entrada (relógio/medidor de energia)?",
        "Notou alguma ocorrência como raios, ventos fortes ou acidentes próximos antes da falta de energia?"
    ],
    "FALTA DE FASE": [
        "Alguns aparelhos funcionam normalmente enquanto outros não ligam?",
        "Todas as tomadas da casa estão funcionando ou apenas algumas?",
        "Você já verificou se todos os disjuntores do quadro de energia estão ligados?",
        "Notou alguma queda de energia ou oscilação antes do problema começar?"
    ],
    "PROVAVEL DEFEITO FALHA": [
        "Onde exatamente você está observando o problema? É na entrada de energia (medidor/poste) ou dentro da residência?",
        "Você verificou todos os disjuntores do quadro de energia interno?",
        "Está ocorrendo algum cheiro de queimado ou ruído estranho? Pode me dizer onde?",
        "O problema começou após alguma tempestade ou ventania?"
    ],
    "NIVEIS DE TENSÃO": [
        "Você percebe lâmpadas piscando ou mais fracas que o normal?",
        "Os aparelhos eletrônicos estão funcionando mais devagar ou desligando sozinhos?",
        "O problema ocorre em toda a vizinhança ou apenas na sua residência?",
        "Já realizou alguma modificação recente na instalação elétrica da sua casa?"
    ],
    "DEFAULT": [
        "Você já verificou o quadro de disjuntores interno da sua residência?",
        "Verificou também o disjuntor geral no padrão de entrada (relógio/medidor)?",
        "Os seus vizinhos estão enfrentando o mesmo problema?",
        "O problema começou após alguma mudança na rede elétrica ou tempestade?"
    ]
}

# Function to determine responsibility based on investigation
def determine_responsibility(issue_type, responses):
    """
    Determine if the problem is internal (client responsibility) or external (distributor responsibility)
    based on the answers to investigation questions.

    Args:
        issue_type: The type of issue reported
        responses: Dictionary of question responses

    Returns:
        is_internal: Boolean indicating if it's an internal problem
        certainty: Level of certainty (high, medium, low)
        reason: Explanation for the decision
    """
    is_internal = False
    certainty = "low"
    reason = "Não foi possível determinar com certeza se o problema é interno ou externo."

    # Check for clear indicators in the responses
    if "vizinhos" in responses and responses["vizinhos"]:
        if "não" in responses["vizinhos"].lower() and "também" not in responses["vizinhos"].lower():
            # Only client affected, likely internal
            is_internal = True
            certainty = "high"
            reason = "Apenas a sua residência está sem energia, enquanto os vizinhos têm energia normal."
        elif "sim" in responses["vizinhos"].lower() or "também" in responses["vizinhos"].lower():
            # Neighbors also affected, external problem
            is_internal = False
            certainty = "high"
            reason = "Os vizinhos também estão enfrentando o mesmo problema, indicando um problema na rede de distribuição."

    # Check disjunctor status
    if "disjuntores" in responses and responses["disjuntores"]:
        if "caído" in responses["disjuntores"].lower() or "desarmado" in responses["disjuntores"].lower() or "desligado" in responses["disjuntores"].lower():
            is_internal = True
            certainty = "high"
            reason = "Há disjuntores desarmados no quadro interno, o que indica um problema na instalação interna."

    # If problem is in the external meter/pole
    if "localização" in responses and responses["localização"]:
        if "poste" in responses["localização"].lower() or "rua" in responses["localização"].lower() or "antes do medidor" in responses["localização"].lower():
            is_internal = False
            certainty = "high"
            reason = "O problema está localizado no poste, na rua ou antes do medidor, que são responsabilidade da distribuidora."
        elif "dentro" in responses["localização"].lower() or "após o medidor" in responses["localização"].lower() or "depois do relógio" in responses["localização"].lower():
            is_internal = True
            certainty = "high"
            reason = "O problema está localizado dentro da residência ou após o medidor/relógio, que é responsabilidade do cliente."

    # If there are visible damages to the distributor's equipment
    if "danos_visíveis" in responses and responses["danos_visíveis"]:
        if "poste" in responses["danos_visíveis"].lower() or "fio caído" in responses["danos_visíveis"].lower() or "transformador" in responses["danos_visíveis"].lower():
            is_internal = False
            certainty = "high"
            reason = "Há danos visíveis na rede externa (poste, fios ou transformador), que são responsabilidade da distribuidora."

    # If the investigation wasn't conclusive, make educated guess based on the issue type
    if certainty == "low":
        if issue_type == "RISCO DE MORTE":
            is_internal = False
            certainty = "medium"
            reason = "Situações de risco na rede elétrica geralmente são responsabilidade da distribuidora."
        elif issue_type == "EMERGENCIA MEDICA":
            is_internal = False
            certainty = "medium"
            reason = "Emergências médicas envolvendo energia são tratadas como prioridade da distribuidora."

    return is_internal, certainty, reason

# Function to generate a unique protocol number
def generate_protocol():
    timestamp = int(time.time())
    return f"NEO-{timestamp}-{int(timestamp/13) % 1000:03d}"

# Function to send message to AI model
def send_to_ai(prompt, conversation_history=None):
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    system_prompt = """Você é um atendente virtual da Neoenergia Elektro, uma distribuidora de energia elétrica.
    Seu nome é Maria e você deve atender aos clientes com muita empatia e compreensão.

    IMPORTANTE:
    1. Você deve investigar adequadamente para determinar se o problema relatado é de responsabilidade da distribuidora ou do cliente
    2. Faça perguntas específicas para distinguir entre problemas internos (após o medidor) e externos (rede da distribuidora)
    3. SEMPRE peça ao cliente se ele já verificou os disjuntores do quadro interno e do padrão de entrada antes de concluir o diagnóstico
    4. Gere protocolo de atendimento APENAS para problemas que são responsabilidade da distribuidora
    5. Para problemas internos (após o medidor/relógio), oriente o cliente mas NÃO gere protocolo

    Seja sempre cordial, paciente e demonstre preocupação genuína com o problema do cliente.
    Use uma linguagem clara e acessível, evitando termos técnicos complexos.
    Forneça orientações de segurança quando necessário.
    """

    messages = [{"role": "system", "content": system_prompt}]

    # Add conversation history if provided
    if conversation_history:
        messages.extend(conversation_history)

    # Add the current prompt
    messages.append({"role": "user", "content": prompt})

    data = {
        "model": MODEL,
        "messages": messages,
        "temperature": 0.7,
        "max_tokens": 800
    }

    try:
        response = requests.post(URL, headers=headers, data=json.dumps(data))
        if response.status_code == 200:
            return response.json()["choices"][0]["message"]["content"]
        else:
            return f"Erro na API: {response.status_code} - {response.text}"
    except Exception as e:
        return f"Erro ao conectar com a API: {str(e)}"

# Updated main function for interactive investigation
def handle_customer_complaint():
    print("=== Atendimento Virtual Neoenergia Elektro ===")
    print("Digite 'sair' para encerrar o atendimento\n")

    # Initial greeting
    print("Maria: Olá! Sou Maria, assistente virtual da Neoenergia Elektro. Como posso ajudar você hoje?")

    conversation_history = []
    investigation_complete = False
    is_internal_problem = None

    while True:
        user_input = input("\nVocê: ")
        if user_input.lower() in ['sair', 'exit', 'quit']:
            print("\nMaria: Obrigada por entrar em contato com a Neoenergia Elektro. Tenha um ótimo dia!")
            break

        # Store the user message in history
        conversation_history.append({"role": "user", "content": user_input})

        # First message - detect initial issue type
        if len(conversation_history) == 1:
            # Get initial issue assessment
            initial_issue_type = identify_initial_issue_type(user_input)

            # Generate investigation prompt based on the issue type
            investigation_questions = INVESTIGATION_QUESTIONS.get(
                initial_issue_type, INVESTIGATION_QUESTIONS["DEFAULT"]
            )

            investigation_prompt = f"""
O cliente relatou: "{user_input}"

Com base nesse relato inicial, identifiquei que pode ser um problema de {initial_issue_type}.

IMPORTANTE: Antes de determinar se é um problema interno ou da distribuidora, preciso investigar melhor.

Por favor, faça as seguintes perguntas ao cliente de forma conversacional (uma de cada vez, não liste todas):
1. Pergunte se o problema afeta apenas a residência dele ou também os vizinhos
2. Pergunte se o cliente já verificou os disjuntores no quadro interno da residência
3. Pergunte se verificou o disjuntor geral no padrão de entrada (medidor/relógio)
4. Pergunte se há algum sinal visível de problema (como fios caídos, cheiro de queimado, etc.)

Seja empático e explique que essas perguntas ajudarão a identificar o tipo de problema e direcionar o melhor atendimento.
"""

            response = send_to_ai(investigation_prompt)

        # Investigation phase
        elif not investigation_complete:
            # Get more context about previous conversation
            context_prompt = f"""
Analise a conversa atual com o cliente e continue a investigação para determinar se o problema é interno (responsabilidade do cliente) ou externo (responsabilidade da distribuidora).

Mensagem mais recente do cliente: "{user_input}"

IMPORTANTE:
1. Se o cliente respondeu a uma pergunta sobre verificação de disjuntores, vizinhos afetados, ou localização do problema, use essa informação
2. Se ainda não tiver perguntado sobre os disjuntores do quadro interno e do padrão de entrada, pergunte isso agora
3. Se já tiver informações suficientes para determinar se é problema interno ou externo, conclua a investigação e:
   a) Para problemas internos: Explique ao cliente que é responsabilidade dele e oriente sobre os próximos passos
   b) Para problemas da distribuidora: Gere um protocolo e explique o processo de atendimento

Mantenha a conversa natural e empática.
"""

            # Add the full conversation history
            full_history = []
            for msg in conversation_history:
                full_history.append(msg)

            response = send_to_ai(context_prompt, full_history[:-1])  # Excluding the last message which we included in the prompt

            # Check if the response indicates completion of investigation
            if "protocolo" in response.lower() or "eletricista particular" in response.lower() or "responsabilidade do cliente" in response.lower() or "nossa equipe" in response.lower():
                investigation_complete = True

        # After investigation is complete
        else:
            follow_up_prompt = f"""
A investigação do problema já foi concluída. O cliente está perguntando: "{user_input}"

Responda à pergunta do cliente mantendo a coerência com o que já foi determinado anteriormente sobre o tipo de problema e quem é o responsável.

Se o cliente estiver perguntando sobre o status do atendimento ou outra informação relacionada, forneça as orientações adequadas.
"""
            response = send_to_ai(follow_up_prompt, conversation_history[:-1])

        # Store assistant's response in history
        conversation_history.append({"role": "assistant", "content": response})

        print(f"\nMaria: {response}")

# Function to identify initial issue type from first message
def identify_initial_issue_type(user_input):
    user_input = user_input.upper()

    # First try to find exact matches
    for type_name, subtypes in ISSUE_TYPES.items():
        for subtype in subtypes:
            if subtype.upper() in user_input:
                return type_name

    # If no exact match, look for keywords
    keywords = {
        "FALTA DE ENERGIA": ["sem luz", "acabou a luz", "energia caiu", "não tem energia", "sem energia", "sem eletricidade"],
        "RISCO DE MORTE": ["fio caído", "fio na rua", "poste quebrado", "fio partido", "risco", "perigo", "acidente"],
        "PROVAVEL DEFEITO FALHA": ["faísca", "curto", "queimando", "cheiro", "defeito", "falha", "problema", "quebrou"],
        "NIVEIS DE TENSÃO": ["tensão", "oscilação", "baixa voltagem", "alta voltagem", "queimou aparelho", "piscando"],
        "EMERGENCIA MEDICA": ["hospital", "clínica", "aparelho médico", "respirador", "equipamento", "vida", "paciente"],
        "OUTROS": ["transformador", "caixa de luz", "poste"]
    }

    for type_name, type_keywords in keywords.items():
        for keyword in type_keywords:
            if keyword.upper() in user_input:
                return type_name

    # Default if no match found
    return "FALTA DE ENERGIA"

# Class-based implementation for more structured conversation flow
class NeoEnergiaAssistant:
    def __init__(self):
        self.conversation_history = []
        self.investigation_stage = 0  # 0: initial, 1-4: investigation questions, 5: conclusion
        self.issue_type = None
        self.is_internal_problem = None
        self.investigation_responses = {}
        self.protocol = None

    def reset(self):
        self.__init__()

    def process_message(self, user_input):
        # Add user message to history
        self.conversation_history.append({"role": "user", "content": user_input})

        # Initial greeting or first message
        if self.investigation_stage == 0:
            # Identify issue type
            self.issue_type = identify_initial_issue_type(user_input)
            self.investigation_stage = 1

            # First investigation question
            investigation_questions = INVESTIGATION_QUESTIONS.get(
                self.issue_type, INVESTIGATION_QUESTIONS["DEFAULT"]
            )

            initial_prompt = f"""
O cliente relatou: "{user_input}"

Baseado nisso, parece ser um problema de {self.issue_type}.

Responda ao cliente com empatia e faça esta pergunta específica para iniciar a investigação:
- {investigation_questions[0]}

Explique brevemente que você precisa fazer algumas perguntas para entender melhor a situação e poder ajudar da melhor forma.
"""
            response = send_to_ai(initial_prompt)

        # Investigation questions (stages 1-4)
        elif 1 <= self.investigation_stage <= 4:
            # Store response to previous question
            question_keys = ["vizinhos", "disjuntores", "disjuntor_geral", "danos_visíveis"]
            self.investigation_responses[question_keys[self.investigation_stage-1]] = user_input

            # Get next question or conclude investigation
            investigation_questions = INVESTIGATION_QUESTIONS.get(
                self.issue_type, INVESTIGATION_QUESTIONS["DEFAULT"]
            )

            if self.investigation_stage < 4:
                next_question = investigation_questions[self.investigation_stage]

                next_prompt = f"""
O cliente respondeu: "{user_input}"

Continue a investigação de forma conversacional. Agradeça pela resposta e faça a próxima pergunta:
- {next_question}
"""
                response = send_to_ai(next_prompt)

In [ ]:
import requests

# Configurações da API
API_KEY = "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"  # Sua chave do Groq
MODEL = "llama3-70b-8192"  # Modelo Llama 3 70B
URL = "https://api.groq.com/openai/v1/chat/completions"

def atendimento_elektro():
    print("\n🔵 Bem-vindo ao Atendimento Neoenergia Elektro! Como posso ajudar você hoje?")
    print("(Digite 'sair' a qualquer momento para encerrar o atendimento)\n")

    historico = [
        {
            "role": "system",
            "content": """Você é um atendente virtual da Neoenergia Elektro, especializado em resolver problemas de fornecimento de energia.
            Seu papel é identificar o problema do cliente com empatia e clareza, coletando informações precisas para resolver a situação.
            Sempre seja educado, paciente e mostre preocupação genuína com a situação do cliente.
            Pergunte de forma clara para identificar o tipo de problema (falta de energia, risco de morte, defeitos, etc.) e subtipo específico.
            Para problemas graves como 'risco de morte' ou 'emergência médica', priorize imediatamente e informe que acionará a equipe de emergência.
            Para outros problemas, explique o procedimento padrão e o prazo estimado para resolução."""
        }
    ]

    while True:
        # Pega a mensagem do cliente
        mensagem_cliente = input("Cliente: ")

        if mensagem_cliente.lower() in ["sair", "exit"]:
            print("\n🟡 Obrigado por entrar em contato com a Neoenergia Elektro. Tenha um bom dia!")
            break

        # Adiciona ao histórico
        historico.append({"role": "user", "content": mensagem_cliente})

        try:
            # Envia para a API
            resposta = requests.post(
                URL,
                headers={"Authorization": f"Bearer {API_KEY}"},
                json={
                    "model": MODEL,
                    "messages": historico,
                    "temperature": 0.5  # Menor variação para respostas mais consistentes
                }
            )

            # Processa a resposta
            if resposta.status_code == 200:
                resposta_ia = resposta.json()["choices"][0]["message"]["content"]
                print(f"\nAtendente Elektro: {resposta_ia}")
                historico.append({"role": "assistant", "content": resposta_ia})
            else:
                print(f"\n🔴 Desculpe, houve um erro em nosso sistema. Por favor, tente novamente mais tarde.")

        except Exception as e:
            print(f"\n🔴 Desculpe, estamos com problemas técnicos. Por favor, tente novamente ou ligue para nosso SAC: 0800 701 0102")

# Inicia o atendimento
if __name__ == "__main__":
    atendimento_elektro()


🔵 Bem-vindo ao Atendimento Neoenergia Elektro! Como posso ajudar você hoje?
(Digite 'sair' a qualquer momento para encerrar o atendimento)

Cliente: então meu amigo eu estou com um problema aqui que o meu chuveiro não está ligando, o restante da casa está tudo normal mas o chuveiro não esquenta

Atendente Elektro: Peço desculpas pelo incoveniente, senhor! Estou aqui para ajudá-lo a resolver o problema com o chuveiro. Posso imaginar como é frustrante não ter água quente para o seu banho.

Para melhor entender a situação, posso perguntar algumas coisas? Você notou se há alguma luz ou indicação de funcionamento no chuveiro? Ou se há algum som ou barulho quando você tenta ligá-lo?

Além disso, você já verificou se o problema é apenas com o chuveiro ou se há alguma outra parte da casa que também esteja afetada? Por exemplo, se as luzes ou outros aparelhos elétricos estão funcionando normalmente?

Com essas informações, posso começar a investigar o problema e encontrar a solução mais adequa

In [ ]:
import requests
from datetime import datetime

# Configurações da API Groq
API_KEY = "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"
MODEL = "llama3-70b-8192"
URL = "https://api.groq.com/openai/v1/chat/completions"

# Dicionário de problemas e classificações
PROBLEMAS = {
    "FALTA DE ENERGIA": [
        "FALTA DE ENERGIA GERAL", "FALTA DE ENERGIA", "FALTA DE FASE",
        "SIGFI FALTA DE ENERGIA FOTOVOLTAICO", "FALTA DE FASE GERAL",
        "Falta de Energia em Medidor Eletronico", "Falta de Fase em Medidor Eletrônico"
    ],
    "RISCO DE MORTE": [
        "OUTROS", "FIO PARTIDO NA REDE PRIMARIA", "FIO PARTIDO NO RAMAL DE SERVICO",
        "FIO PARTIDO NA REDE SECUNDARIA", "POSTE ABALROADO SEM FIO PARTIDO",
        "POSTE ABALROADO COM FIO PARTIDO"
    ],
    "PROVAVEL DEFEITO FALHA": [
        "FURTO FIACÃO/PADRÃO", "FALHA EM RAMAL DE SERVICO", "FIO DANDO FOGO NO RAMAL DE SERVICO",
        "FALHA EM MEDIDOR", "OBJETO ESTRANHO NA REDE SECUNDARIA", "OBJETO ESTRANHO NA REDE PRIMARIA",
        "POSTE PADRÃO QUEBRADO", "FIO DANDO FOGO NA REDE PRIMARIA", "FIO DANDO FOGO COM ARVORE NA REDE",
        "FURTO EQUIPAMENTO", "ELO FUSIVEL QUEIMADO/CAIDO", "FIO DANDO FOGO NA REDE SECUNDARIA",
        "CHOQUE EM INSTALAÇÃO", "FALHA EM QUICK-LAG", "FURTO REDE", "INVERSÃO DE FASE"
    ],
    "NIVEIS DE TENSÃO": [
        "TENSÃO BAIXA", "OSCILAÇÃO DE TENSÃO", "TENSÃO ALTA"
    ],
    "OUTROS": [
        "TRANSFORMADOR"
    ],
    "EMERGENCIA MEDICA": [
        "HOSPITAL COM PROBLEMA", "EQUIPAMENTO SUPORTE A VIDA"
    ]
}

def obter_saudacao():
    """Retorna a saudação conforme horário atual"""
    hora = datetime.now().hour
    if 5 <= hora < 12:
        return "Bom dia"
    elif 12 <= hora < 18:
        return "Boa tarde"
    return "Boa noite"

def obter_mensagem_fechamento():
    """Retorna mensagem padrão de encerramento"""
    return """
Agradecemos seu contato! Sempre que precisar estamos disponíveis nos nossos canais:

💻 Site: https://www.neoenergia.com/web/sp

📱 Aplicativo:
Android → https://bit.ly/NELK-Android
IOS → https://bit.ly/IOS-NELK

📞 Teleatendimento: 116
Deficiente auditivo ou de fala | 0800 701 01 55
Atendimento fora da área de concessão | 0800 276 0116

Conte sempre com a gente!
"""

def classificar_problema(descricao):
    """Classifica o problema em tipo e subtipo"""
    descricao = descricao.upper()
    for tipo, subtipos in PROBLEMAS.items():
        for subtipo in subtipos:
            if subtipo in descricao:
                return tipo, subtipo

    # Fallback para palavras-chave
    if "FALTA" in descricao and "ENERGIA" in descricao:
        return "FALTA DE ENERGIA", "FALTA DE ENERGIA"
    elif "FOGO" in descricao or "INCÊNDIO" in descricao:
        return "RISCO DE MORTE", "FIO DANDO FOGO NA REDE PRIMARIA"
    elif "CHOQUE" in descricao:
        return "PROVAVEL DEFEITO FALHA", "CHOQUE EM INSTALAÇÃO"
    elif "TENSÃO" in descricao or "VOLTAGEM" in descricao:
        return "NIVEIS DE TENSÃO", "OSCILAÇÃO DE TENSÃO"

    return "OUTROS", "OUTROS"

def verificar_problema_interno(descricao):
    """Verifica se problema é na instalação do cliente"""
    descricao = descricao.lower()
    indicadores = [
        "minha casa", "minha instalação", "meu disjuntor", "meu medidor",
        "apenas aqui", "só na minha casa", "circuito interno", "fiação interna",
        "disjuntor caiu", "queimou meu equipamento", "meu chuveiro", "minhas lâmpadas"
    ]
    return any(ind in descricao for ind in indicadores)

def gerar_resposta(tipo, subtipo, problema_interno):
    """Gera resposta adequada ao tipo de problema"""
    if problema_interno:
        return {
            "resposta": "Entendi que o problema parece ser na sua instalação interna. Recomendamos contatar um eletricista qualificado. Posso ajudar em algo mais?",
            "gerar_chamado": False
        }

    respostas = {
        "FALTA DE ENERGIA": {
            "mensagem": "Entendi que há falta de energia. Vou registrar um chamado para nossa equipe verificar.",
            "urgencia": "ALTA"
        },
        "RISCO DE MORTE": {
            "mensagem": "Situação de risco identificada! Equipe será acionada com urgência. Mantenha distância.",
            "urgencia": "URGENTE"
        },
        "PROVAVEL DEFEITO FALHA": {
            "mensagem": "Relato de defeito na rede entendido. Registrarei um chamado técnico.",
            "urgencia": "MEDIA"
        },
        "NIVEIS DE TENSÃO": {
            "mensagem": "Problema de tensão identificado. Registrarei um chamado para ajuste.",
            "urgencia": "MEDIA"
        },
        "EMERGENCIA MEDICA": {
            "mensagem": "Emergência médica! Acionaremos prioridade máxima.",
            "urgencia": "URGENTE"
        },
        "OUTROS": {
            "mensagem": "Entendi seu relato. Vou registrar um chamado para verificação.",
            "urgencia": "BAIXA"
        }
    }

    resposta = respostas.get(tipo, respostas["OUTROS"])
    return {
        "resposta": resposta["mensagem"],
        "gerar_chamado": True,
        "tipo": tipo,
        "subtipo": subtipo,
        "urgencia": resposta["urgencia"]
    }

def enviar_para_groq(mensagem):
    """Envia mensagem para API Groq e retorna resposta"""
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": MODEL,
        "messages": [
            {
                "role": "system",
                "content": "Você é um atendente virtual da Neoenergia Elektro. Seja empático e objetivo. Responda em português brasileiro de forma clara e curta."
            },
            {
                "role": "user",
                "content": mensagem
            }
        ],
        "temperature": 0.7,
        "max_tokens": 150
    }

    try:
        response = requests.post(URL, headers=headers, json=payload)
        response.raise_for_status()
        return response.json()["choices"][0]["message"]["content"]
    except Exception as e:
        return f"Desculpe, estou com dificuldades técnicas. Por favor, tente novamente mais tarde."

# Fluxo principal de atendimento
if __name__ == "__main__":
    print(f"{obter_saudacao()}! Sou o assistente virtual da Neoenergia Elektro. Como posso ajudar?")
    print("(Digite 'sair' para encerrar)")

    while True:
        mensagem = input("\nCliente: ").strip()

        if mensagem.lower() == 'sair':
            print(obter_mensagem_fechamento())
            break

        # Processa mensagem do cliente
        problema_interno = verificar_problema_interno(mensagem)
        tipo, subtipo = classificar_problema(mensagem)
        resposta = gerar_resposta(tipo, subtipo, problema_interno)

        print("\nAtendente:", resposta["resposta"])
        if resposta["gerar_chamado"]:
            print(f"(Chamado: {tipo} - {subtipo} | Urgência: {resposta['urgencia']})")

        # Resposta complementar da IA
        print("Atendente:", enviar_para_groq(mensagem))

Boa tarde! Sou o assistente virtual da Neoenergia Elektro. Como posso ajudar?
(Digite 'sair' para encerrar)

Cliente: Olá eu  estou com um problema, estou sem energia aqui em casa e pelo que eu vi tem uma chave lá no poste que caiu

Atendente: Entendi seu relato. Vou registrar um chamado para verificação.
(Chamado: OUTROS - OUTROS | Urgência: BAIXA)
Atendente: Desculpe, estou com dificuldades técnicas. Por favor, tente novamente mais tarde.

Cliente: então meu amigo eu estou com um problema aqui que o meu chuveiro não está ligando, o restante da casa está tudo normal mas o chuveiro não esquenta

Atendente: Entendi que o problema parece ser na sua instalação interna. Recomendamos contatar um eletricista qualificado. Posso ajudar em algo mais?
Atendente: Poxa, que chato! Não ter água quente no chuveiro pode ser muito inconveniente. 

Para ajudá-lo, gostaria de saber mais algumas coisas. Você verifica se o botão do chuveiro está ligado e se a chave de luz do banheiro está funcionando corr

**CÓDIGO ATUAL**

In [ ]:
import requests
from datetime import datetime
import json

# Configurações da API Groq
API_KEY = "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"
MODEL = "llama3-70b-8192"
GROQ_URL = "https://api.groq.com/openai/v1/chat/completions"

# Tabela de categorização de problemas (simplificada para o contexto)
PROBLEMS_TABLE = {
    "emergencia_medica": {
        "descricao": "Falta de energia afetando equipamentos de suporte à vida",
        "criticidade": "Emergência",
        "acao": "Atendimento imediato com prioridade máxima"
    },
    "falta_energia": {
        "descricao": "Casa do cliente totalmente sem energia",
        "criticidade": "Médio",
        "acao": "Inspeção técnica e correção"
    },
    "falta_energia_area": {
        "descricao": "Interrupção total de energia em uma área ampla",
        "criticidade": "Alto",
        "acao": "Manutenção corretiva urgente"
    },
    "falta_fase": {
        "descricao": "Falta de uma das fases da rede elétrica",
        "criticidade": "Médio",
        "acao": "Manutenção corretiva"
    },
    "tensao_alta": {
        "descricao": "Voltagem acima do normal",
        "criticidade": "Alto",
        "acao": "Inspeção técnica urgente"
    },
    "tensao_baixa": {
        "descricao": "Voltagem abaixo do normal",
        "criticidade": "Médio",
        "acao": "Inspeção técnica"
    },
    "choque": {
        "descricao": "Cliente relata choque ao tocar em instalações",
        "criticidade": "Emergência",
        "acao": "Inspeção técnica imediata"
    },
    "fio_partido": {
        "descricao": "Cabo rompido na rede",
        "criticidade": "Emergência",
        "acao": "Isolamento imediato da área e reparo urgente"
    },
    "fogo_rede": {
        "descricao": "Fio apresentando faíscas ou princípio de incêndio",
        "criticidade": "Emergência",
        "acao": "Atendimento imediato"
    }
}

def get_current_greeting():
    """Retorna a saudação de acordo com o horário atual"""
    hour = datetime.now().hour
    if 5 <= hour < 12:
        return "Bom dia"
    elif 12 <= hour < 18:
        return "Boa tarde"
    else:
        return "Boa noite"

def generate_farewell():
    """Retorna a mensagem de encerramento padrão"""
    return """
Agradecemos seu contato! Sempre que precisar estamos disponíveis nos nossos canais:
💻 Site: https://www.neoenergia.com/web/sp

📱 Aplicativo:
Android → https://bit.ly/NELK-Android
IOS → https://bit.ly/IOS-NELK

📞 Teleatendimento: 116
Deficiente auditivo ou de fala | 0800 701 01 55
Atendimento fora da área de concessão | 0800 276 0116

Conte sempre com a gente!
"""

def analyze_problem(client_message):
    """Analisa a mensagem do cliente para identificar o problema"""
    message_lower = client_message.lower()

    # Verifica problemas críticos primeiro
    if any(term in message_lower for term in ["equipamento de suporte à vida", "respirador", "máquina de oxigênio"]):
        return "emergencia_medica"
    elif any(term in message_lower for term in ["fio partido", "cabo quebrado", "cabo rompido"]):
        return "fio_partido"
    elif any(term in message_lower for term in ["fogo no fio", "faísca", "fogo na rede", "fiação pegando fogo"]):
        return "fogo_rede"
    elif any(term in message_lower for term in ["tomando choque", "levando choque", "eletrocutando"]):
        return "choque"
    elif any(term in message_lower for term in ["tensão alta", "luz muito forte", "aparelhos queimando"]):
        return "tensao_alta"
    elif any(term in message_lower for term in ["tensão baixa", "luz fraca", "aparelhos não funcionam direito"]):
        return "tensao_baixa"
    elif any(term in message_lower for term in ["falta fase", "algumas lâmpadas acendem", "só um lado tem luz"]):
        return "falta_fase"
    elif any(term in message_lower for term in ["vizinhança sem luz", "rua inteira sem energia", "apagão"]):
        return "falta_energia_area"
    elif any(term in message_lower for term in ["sem energia", "sem luz", "queda de energia"]):
        return "falta_energia"
    else:
        return None

def check_internal_problem(client_message):
    """Verifica se o problema pode ser interno"""
    message_lower = client_message.lower()
    internal_terms = [
        "só na minha casa", "apenas aqui", "vizinhos têm luz",
        "disjuntor caiu", "disjuntor desarmou", "fusível queimou",
        "queda interna", "problema no quadro"
    ]
    return any(term in message_lower for term in internal_terms)

def generate_response(client_message, problem_type=None):
    """Gera uma resposta apropriada baseada no tipo de problema"""
    if problem_type is None:
        problem_type = analyze_problem(client_message)

    greeting = get_current_greeting()

    if problem_type is None:
        # Se não identificou um problema específico
        return f"{greeting}! Para melhor atendê-lo, poderia descrever com mais detalhes o problema que está enfrentando com a energia elétrica?"

    problem_info = PROBLEMS_TABLE.get(problem_type, {})

    if check_internal_problem(client_message):
        return f"{greeting}! Pelo que descreveu, parece ser um problema na instalação interna. Recomendo verificar:\n1. Se o disjuntor principal não desarmou\n2. Se há fusíveis queimados\n3. Se os disjuntores individuais estão ativados\nCaso persista, pode ser necessário um eletricista qualificado."

    if problem_type in PROBLEMS_TABLE:
        if problem_info["criticidade"] == "Emergência":
            # Gerar ticket de emergência
            ticket_id = f"EMG-{datetime.now().strftime('%Y%m%d%H%M%S')}"
            return f"{greeting}! Identificamos uma situação de emergência. Já estamos acionando nossa equipe técnica com prioridade máxima. Seu protocolo é {ticket_id}. Por segurança, mantenha distância do local e evite tocar em qualquer parte elétrica."
        else:
            # Gerar ticket normal
            ticket_id = f"CHM-{datetime.now().strftime('%Y%m%d%H%M%S')}"
            return f"{greeting}! Entendemos o problema e já estamos registrando sua ocorrência sob o protocolo {ticket_id}. Nossa equipe será acionada para resolver o mais breve possível."
    else:
        return f"{greeting}! Vamos analisar seu relato e encontrar a melhor solução. Poderia confirmar se o problema afeta apenas sua residência ou toda a vizinhança?"

def call_groq_api(client_message, conversation_history=[]):
    """Chama a API do Groq para gerar uma resposta mais contextualizada"""
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    # Construir o prompt com o contexto do atendimento
    prompt = f"""
Você é um assistente virtual da Neoenergia Elektro, uma distribuidora de energia elétrica.
Seu papel é atender clientes com empatia e eficiência, identificando problemas elétricos e orientando sobre soluções.

Contexto:
- Analise a mensagem do cliente para identificar o tipo de problema
- Se for um problema interno (da instalação do cliente), oriente sobre como verificar disjuntores e fusíveis
- Se for um problema externo (da rede elétrica), gere um ticket de atendimento
- Se for emergência (equipamento médico, fio partido, choque elétrico), priorize o atendimento
- Seja breve, objetivo e empático
- A mensagem deve ter no máximo 2 parágrafos curtos

Histórico da conversa:
{conversation_history}

Mensagem do cliente: {client_message}
"""

    data = {
        "model": MODEL,
        "messages": [
            {"role": "system", "content": prompt},
            {"role": "user", "content": client_message}
        ],
        "temperature": 0.7,
        "max_tokens": 150
    }

    try:
        response = requests.post(GROQ_URL, headers=headers, json=data)
        response.raise_for_status()
        return response.json()["choices"][0]["message"]["content"]
    except Exception as e:
        print(f"Erro ao chamar API Groq: {e}")
        return generate_response(client_message)

def main():
    """Função principal para simular o atendimento"""
    print(f"{get_current_greeting()}! Sou o assistente virtual da Neoenergia Elektro. Como posso ajudar você hoje?")

    conversation_history = []

    while True:
        client_input = input("Cliente: ").strip()

        if client_input.lower() in ["sair", "encerrar", "adeus", "obrigado"]:
            print(generate_farewell())
            break

        # Adiciona ao histórico (limitando o tamanho)
        conversation_history.append(f"Cliente: {client_input}")
        if len(conversation_history) > 6:  # Mantém apenas as últimas 3 interações
            conversation_history = conversation_history[-6:]

        # Obtém resposta
        response = call_groq_api(client_input, conversation_history)

        print("Assistente:", response)

        # Adiciona resposta ao histórico
        conversation_history.append(f"Assistente: {response}")

if __name__ == "__main__":
    main()

Boa tarde! Sou o assistente virtual da Neoenergia Elektro. Como posso ajudar você hoje?
Cliente: olá eu estou sem uma das fases aqui em casa, parece que caiu a banana lá do poste
Assistente: Peço desculpas pelo incoveniente! É possível que haja um problema na rede elétrica externa. Vou gerar um ticket de atendimento para que nossa equipe técnica possa verificar a situação e resolver o problema o mais rápido possível. Por favor, aguarde um momento enquanto processo o seu pedido. Em breve, você receberá um retorno com mais informações sobre o atendimento.
Cliente: agora estou com um problema as lampadas da rua estão apagadas
Assistente: Peço desculpas pelo novo problema! É provável que haja um problema na rede elétrica externa novamente. Vou gerar outro ticket de atendimento para que nossa equipe técnica possa verificar a situação e resolver o problema o mais rápido possível. Por favor, aguarde um momento enquanto processo o seu pedido. Em breve, você receberá um retorno com mais informa

KeyboardInterrupt: Interrupted by user

**CÓDIGO CLAUDE**

In [ ]:
!pip install python-dotenv

In [ ]:
import requests
import json
import datetime
import re
import os
from dotenv import load_dotenv

# Carregar variáveis de ambiente
load_dotenv()

# Configurações da API - Corrigindo o modo de acesso à API
API_KEY = os.getenv("GROQ_API_KEY", "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")  # Fallback para a chave fornecida
MODEL = "llama3-70b-8192"
URL = "https://api.groq.com/openai/v1/chat/completions"

# Base de conhecimento da Neoenergia Elektro
CATEGORIAS_PROBLEMAS = [
    {
        "categoria": "Técnico",
        "tipo": "EMERGENCIA MEDICA",
        "subtipo": "EQUIPAMENTO SUPORTE A VIDA",
        "descricao": "Falta de energia afetando equipamentos de suporte à vida",
        "causas": ["Falha na rede", "problema no fornecimento local"],
        "criticidade": "Emergência",
        "acao": "Atendimento imediato com prioridade máxima",
        "verificacoes": ["Verificar se há gerador auxiliar funcionando"],
        "termos": ["equipamento de suporte à vida", "respirador", "máquina de oxigênio"],
        "seguranca": "Acionar serviço médico de emergência simultaneamente",
        "interno": False
    },
    {
        "categoria": "Técnico",
        "tipo": "EMERGENCIA MEDICA",
        "subtipo": "HOSPITAL COM PROBLEMA",
        "descricao": "Falta de energia em hospital",
        "causas": ["Falha na rede", "problema no gerador"],
        "criticidade": "Emergência",
        "acao": "Atendimento imediato",
        "verificacoes": ["Verificar se o hospital tem gerador funcionando"],
        "termos": ["hospital está sem luz", "energia para um hospital"],
        "seguranca": "Solicitar prioridade ao informar a ocorrência",
        "interno": False
    },
    {
        "categoria": "Técnico",
        "tipo": "Falta de Energia",
        "subtipo": "Falta de Energia",
        "descricao": "Casa do cliente totalmente sem energia",
        "causas": ["Falha no ramal do cliente", "falha interna", "disjuntores desarmado",
                  "problema no medidor", "chave fusível desarmada"],
        "criticidade": "Médio",
        "acao": "Inspeção técnica e correção",
        "verificacoes": ["Conferir se disjuntores estão ligados", "testar outros aparelhos"],
        "termos": ["casa está sem luz", "só aqui em casa que caiu a energia"],
        "seguranca": "Desligar aparelhos eletrônicos",
        "interno": True
    },
    {
        "categoria": "Técnico",
        "tipo": "FALTA DE ENERGIA",
        "subtipo": "Falta de Energia em Medidor Eletrônico",
        "descricao": "Medidor eletrônico sem leitura ou funcionamento",
        "causas": ["Falha no medidor", "problema de comunicação", "bateria interna esgotada"],
        "criticidade": "Médio",
        "acao": "Inspeção técnica especializada",
        "verificacoes": ["Verificar se o display do medidor está aceso", "testar religação de disjuntores"],
        "termos": ["relógio digital está apagado", "medidor novo não está funcionando"],
        "seguranca": "Não tentar abrir ou manipular o medidor",
        "interno": False
    },
    {
        "categoria": "Técnico",
        "tipo": "Falta de Energia",
        "subtipo": "Falta de Energia Geral",
        "descricao": "Interrupção total de energia em uma área ampla",
        "causas": ["Falha em subestação", "curto-circuito na rede", "problema em transformador"],
        "criticidade": "Alto",
        "acao": "Manutenção corretiva urgente",
        "verificacoes": ["Verificar se vizinhos também estão sem luz"],
        "termos": ["faltou luz na rua", "vizinhança inteira sem energia", "apagão"],
        "seguranca": "Evitar usar velas, desligar aparelhos eletrônicos",
        "interno": False
    },
    {
        "categoria": "Técnico",
        "tipo": "FALTA DE ENERGIA",
        "subtipo": "FALTA DE FASE",
        "descricao": "Falta de uma das fases da rede elétrica, causando funcionamento parcial",
        "causas": ["Problema no transformador", "fusível queimado", "rompimento de um dos fios de fase"],
        "criticidade": "Médio",
        "acao": "Manutenção corretiva",
        "verificacoes": ["Testar se algumas tomadas funcionam e outras não"],
        "termos": ["algumas lâmpadas acendem, outras não", "só um lado da casa tem luz"],
        "seguranca": "Evitar o uso de aparelhos de alta potência",
        "interno": False
    },
    {
        "categoria": "Técnico",
        "tipo": "RISCO DE MORTE",
        "subtipo": "FIO PARTIDO NA REDE PRIMARIA",
        "descricao": "Cabo de alta tensão rompido",
        "causas": ["Acidente", "tempestade", "deterioração"],
        "criticidade": "Emergência",
        "acao": "Isolamento imediato da área e reparo urgente",
        "verificacoes": ["Observar cabo rompido em altura"],
        "termos": ["fio grosso da alta tensão está partido", "caiu um cabo de alta tensão"],
        "seguranca": "Manter distância mínima de 30 metros, alertar pessoas próximas",
        "interno": False
    },
    {
        "categoria": "Técnico",
        "tipo": "RISCO DE MORTE",
        "subtipo": "FIO PARTIDO NA REDE SECUNDARIA",
        "descricao": "Cabo de baixa tensão rompido",
        "causas": ["Acidente", "impacto", "deterioração"],
        "criticidade": "Emergência",
        "acao": "Isolamento da área e atendimento urgente",
        "verificacoes": ["Verificar cabo visivelmente partido"],
        "termos": ["fio da luz caiu", "cabo quebrado na rua"],
        "seguranca": "Manter distância mínima de 10 metros, alertar vizinhos",
        "interno": False
    },
    {
        "categoria": "Técnico",
        "tipo": "NIVEIS DE TENSÃO",
        "subtipo": "OSCILACÃO DE TENSÃO",
        "descricao": "Variações constantes na tensão elétrica",
        "causas": ["Sobrecarga momentânea", "falha no regulador", "problema no transformador"],
        "criticidade": "Médio",
        "acao": "Monitoramento e análise técnica",
        "verificacoes": ["Observar luzes piscando", "aparelhos reiniciando sozinhos"],
        "termos": ["luz fica piscando", "aparelhos desligam e ligam sozinhos"],
        "seguranca": "Desligar equipamentos sensíveis, utilizar estabilizadores",
        "interno": False
    },
    {
        "categoria": "Técnico",
        "tipo": "NIVEIS DE TENSÃO",
        "subtipo": "TENSÃO ALTA",
        "descricao": "Voltagem acima do normal, causando risco aos aparelhos",
        "causas": ["Regulador com defeito", "transformador sobrecarregado"],
        "criticidade": "Alto",
        "acao": "Inspeção técnica urgente",
        "verificacoes": ["Verificar se lâmpadas queimam com frequência ou brilham intensamente"],
        "termos": ["luz está muito forte", "aparelhos estão queimando"],
        "seguranca": "Desligar todos os aparelhos eletrônicos imediatamente",
        "interno": False
    },
    {
        "categoria": "Técnico",
        "tipo": "NIVEIS DE TENSÃO",
        "subtipo": "TENSÃO BAIXA",
        "descricao": "Voltagem abaixo do normal, afetando o funcionamento de aparelhos",
        "causas": ["Sobrecarga", "problema no transformador"],
        "criticidade": "Médio",
        "acao": "Inspeção técnica",
        "verificacoes": ["Testar se lâmpadas estão fracas e aparelhos falhando"],
        "termos": ["luz está fraca", "aparelhos não funcionam direito"],
        "seguranca": "Evitar ligar muitos aparelhos ao mesmo tempo",
        "interno": False
    },
    {
        "categoria": "Comercial",
        "tipo": "LIGAÇÃO",
        "subtipo": "Ligação Nova",
        "descricao": "Solicitação de ligação de energia para um novo imóvel",
        "causas": ["Novo cliente", "imóvel recém-construído"],
        "criticidade": "Médio",
        "acao": "Programação para instalação",
        "verificacoes": ["Verificar se há padrão de entrada instalado"],
        "termos": ["preciso de uma nova ligação de energia", "casa ainda não tem luz"],
        "seguranca": "Garantir que a instalação elétrica está regularizada antes da ligação",
        "interno": False
    },
    {
        "categoria": "Comercial",
        "tipo": "RELIGAÇÃO",
        "subtipo": "Religação Normal",
        "descricao": "Restabelecimento de energia após corte por inadimplência ou manutenção",
        "causas": ["Pagamento efetuado", "fim de manutenção programada"],
        "criticidade": "Médio",
        "acao": "Execução conforme prazo estabelecido",
        "verificacoes": ["Conferir se a conta foi paga e religação solicitada"],
        "termos": ["paguei a conta e a luz não voltou", "quero religar a energia"],
        "seguranca": "Esperar o prazo da distribuidora para religação antes de fazer nova solicitação",
        "interno": False
    }
]

def obter_saudacao():
    """Retorna uma saudação baseada no horário atual."""
    hora_atual = datetime.datetime.now().hour

    if 5 <= hora_atual < 12:
        return "Bom dia"
    elif 12 <= hora_atual < 18:
        return "Boa tarde"
    else:
        return "Boa noite"

def mensagem_fechamento():
    """Retorna a mensagem padrão de fechamento."""
    return """Agradecemos seu contato! Sempre que precisar estamos disponíveis nos nossos canais:
💻 Site: https://www.neoenergia.com/web/sp

📱 Aplicativo:
Android → https://bit.ly/NELK-Android
IOS → https://bit.ly/IOS-NELK

📞 Teleatendimento: 116
Deficiente auditivo ou de fala | 0800 701 01 55
Atendimento fora da área de concessão | 0800 276 0116

Conte sempre com a gente!"""

def analisar_problema_interno(mensagem):
    """
    Verifica se a mensagem indica um problema interno baseado em palavras-chave.
    Retorna (é_interno, dicas) onde:
    - é_interno: booleano indicando se é um problema interno
    - dicas: sugestões para verificação se for problema interno
    """
    # Palavras-chave que sugerem problemas internos
    padroes_internos = [
        r"só (em )?minha casa",
        r"(apenas|somente) (a )?minha casa",
        r"disjuntor( geral)? (desarm|caiu|desligou)",
        r"queim(ei|ou) um?a? (fusível|lâmpada)",
        r"geladeira (parou|não funciona)",
        r"(só |apenas |somente )(um cômodo|uma parte)",
        r"(já )?verifiqu?ei com (o vizinho|os vizinhos)",
        r"(não tem|sem) luz (apenas|só|somente) aqui"
    ]

    # Verificar padrões internos na mensagem
    for padrao in padroes_internos:
        if re.search(padrao, mensagem.lower()):
            return True, [
                "Verificar se os disjuntores estão ligados",
                "Testar se a energia funciona em algum ponto da casa",
                "Conferir se os vizinhos também estão sem energia"
            ]

    # Verificar correspondências com os problemas da base de conhecimento
    for problema in CATEGORIAS_PROBLEMAS:
        if problema.get("interno", False):
            for termo in problema.get("termos", []):
                if termo.lower() in mensagem.lower():
                    return True, problema.get("verificacoes", [])

    return False, []

def identificar_categoria_problema(mensagem):
    """
    Identifica a categoria de problema com base na mensagem do cliente.
    Retorna o problema mais relevante ou None se não encontrar.
    """
    relevancia_maxima = 0
    problema_identificado = None

    # Verificar criticidade emergência primeiro
    for problema in CATEGORIAS_PROBLEMAS:
        if problema.get("criticidade") == "Emergência":
            for termo in problema.get("termos", []):
                if termo.lower() in mensagem.lower():
                    return problema

    # Verificar demais problemas
    for problema in CATEGORIAS_PROBLEMAS:
        relevancia = 0
        for termo in problema.get("termos", []):
            if termo.lower() in mensagem.lower():
                relevancia += 1

        if relevancia > relevancia_maxima:
            relevancia_maxima = relevancia
            problema_identificado = problema

    return problema_identificado

def gerar_protocolo():
    """Gera um número de protocolo baseado na data e hora atuais."""
    agora = datetime.datetime.now()
    return f"{agora.year}{agora.month:02d}{agora.day:02d}{agora.hour:02d}{agora.minute:02d}{agora.second:02d}"

def consultar_ia(mensagem_cliente, contexto_anterior=None):
    """
    Consulta a IA para obter resposta ao cliente.

    Args:
        mensagem_cliente: Mensagem do cliente
        contexto_anterior: Histórico de mensagens anteriores (opcional)

    Returns:
        Resposta da IA
    """
    # Determinar se é problema interno
    e_interno, dicas_internas = analisar_problema_interno(mensagem_cliente)

    # Identificar categoria do problema
    problema = identificar_categoria_problema(mensagem_cliente)

    # Compilar informações para enviar à IA
    informacoes = {
        "mensagem_cliente": mensagem_cliente,
        "é_problema_interno": e_interno,
        "dicas_verificacao": dicas_internas,
        "problema_identificado": problema,
        "protocolo": gerar_protocolo() if not e_interno and problema else None
    }

    # Construir conteúdo do problema de forma segura
    problema_info = ""
    if problema:
        problema_info = f"""
        - Categoria: {problema.get('categoria', '')}
        - Tipo: {problema.get('tipo', '')}
        - Subtipo: {problema.get('subtipo', '')}
        - Descrição: {problema.get('descricao', '')}
        - Criticidade: {problema.get('criticidade', '')}
        - Ação necessária: {problema.get('acao', '')}
        - Cuidados de segurança: {problema.get('seguranca', '')}
        """

    # Criar prompt para a IA
    sistema_prompt = f"""
    Você é um atendente virtual da Neoenergia Elektro, uma distribuidora de energia.
    Seu nome é BotNinja e você deve atender ao cliente com empatia e objetividade.

    Informações sobre o problema do cliente:
    - Mensagem: {informacoes['mensagem_cliente']}
    - Problema interno: {"Sim" if informacoes['é_problema_interno'] else "Não"}

    {("- Dicas para verificação: " + ", ".join(informacoes['dicas_verificacao'])) if informacoes['dicas_verificacao'] else ""}

    {problema_info}

    {("- PROTOCOLO GERADO: " + informacoes['protocolo']) if informacoes['protocolo'] else ""}

    Diretrizes para sua resposta:
    1. Seja empático e compreensivo com o problema do cliente
    2. Use linguagem clara e objetiva, evitando textos muito longos
    3. Se for um problema interno, oriente o cliente a verificar os disjuntores e outras possíveis causas internas
    4. Se for um problema externo, informe o cliente sobre o protocolo gerado e os próximos passos
    5. Para problemas de Emergência ou Alto risco, enfatize as medidas de segurança
    6. Não use emojis excessivos nem linguagem muito técnica
    7. Não use "Neoenergia Elektro" em cada frase, apenas uma ou duas vezes na resposta
    8. Em sua resposta, você pode iniciar com "{obter_saudacao()}!"
    9. Não encerre a conversa a menos que o problema tenha sido resolvido
    """

    # Preparar headers e dados para a API
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    messages = [{"role": "system", "content": sistema_prompt}]

    # Adicionar contexto anterior se existir
    if contexto_anterior:
        messages.extend(contexto_anterior)

    # Adicionar mensagem atual do cliente
    messages.append({"role": "user", "content": mensagem_cliente})

    data = {
        "model": MODEL,
        "messages": messages,
        "temperature": 0.7,
        "max_tokens": 500
    }

    try:
        # Fazer requisição para a API - Adicionando mais informações de debug
        print(f"Enviando requisição para: {URL}")
        response = requests.post(URL, headers=headers, json=data)  # Usando json= em vez de data=json.dumps()

        if response.status_code != 200:
            print(f"Erro na API: Status {response.status_code}")
            print(f"Resposta: {response.text}")
            # Fallback para respostas em caso de erro na API
            return criar_resposta_fallback(e_interno, problema, informacoes)

        # Processar resposta
        resposta = response.json()
        conteudo = resposta["choices"][0]["message"]["content"]
        return conteudo

    except Exception as e:
        # Melhor tratamento de exceção com log do erro
        print(f"Erro ao chamar a API: {str(e)}")
        return criar_resposta_fallback(e_interno, problema, informacoes)

def criar_resposta_fallback(e_interno, problema, informacoes):
    """Cria uma resposta de fallback em caso de falha na API"""
    if e_interno:
        return f"{obter_saudacao()}! Identificamos que você pode estar com um problema na instalação interna. Recomendamos verificar os disjuntores e se os vizinhos também estão sem energia. Se o problema persistir, entre em contato novamente."
    elif problema and problema.get("criticidade") == "Emergência":
        return f"{obter_saudacao()}! Atenção! Identificamos uma situação de emergência. Por favor, siga as orientações de segurança: {problema.get('seguranca', 'Mantenha-se seguro e aguarde nossa equipe')}. Geramos o protocolo {informacoes['protocolo']} e uma equipe será enviada com urgência."
    elif problema:
        return f"{obter_saudacao()}! Registramos seu problema de {problema.get('descricao', 'energia')}. Geramos o protocolo {informacoes['protocolo']} para acompanhamento. Nossa equipe irá avaliar a situação o mais breve possível."
    else:
        return f"{obter_saudacao()}! Obrigado por entrar em contato com a Neoenergia Elektro. Para que possamos ajudar melhor, poderia fornecer mais detalhes sobre o problema que está enfrentando?"

def atendimento_cliente():
    """Função principal para iniciar atendimento a cliente."""
    print("\n=== Sistema de Atendimento Neoenergia Elektro ===\n")

    # Inicializar histórico de mensagens
    historico = []

    # Obter primeira mensagem do cliente
    print(f"[Atendente] {obter_saudacao()}! Sou o atendente virtual da Neoenergia Elektro. Como posso ajudar?")

    while True:
        mensagem_cliente = input("\n[Cliente] ")

        if mensagem_cliente.lower() in ["sair", "tchau", "encerrar", "finalizar"]:
            print(f"\n[Atendente] {mensagem_fechamento()}")
            break

        # Consultar IA para resposta
        resposta = consultar_ia(mensagem_cliente, historico)

        # Adicionar ao histórico
        historico.append({"role": "user", "content": mensagem_cliente})
        historico.append({"role": "assistant", "content": resposta})

        print(f"\n[Atendente] {resposta}")

def processar_mensagem_whatsapp(mensagem, numero_cliente, sessao_id=None):
    """
    Função para processar mensagens vindas do WhatsApp.

    Args:
        mensagem: Texto da mensagem do cliente
        numero_cliente: Número do WhatsApp do cliente
        sessao_id: ID da sessão para rastrear conversas (opcional)

    Returns:
        Resposta para enviar ao cliente
    """
    # Aqui implementaria a lógica para recuperar histórico da sessão
    historico = []  # Em produção, recuperaria do banco de dados

    # Consultar IA
    resposta = consultar_ia(mensagem, historico)

    # Salvar histórico (em produção)
    # salvar_historico(sessao_id, mensagem, resposta)

    return resposta

# Função para testar a conexão com a API
def testar_api():
    """Testa a conexão com a API para verificar se está funcionando"""
    print("Testando conexão com a API Groq...")
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    test_data = {
        "model": MODEL,
        "messages": [
            {"role": "system", "content": "Você é um assistente útil."},
            {"role": "user", "content": "Olá, está funcionando?"}
        ],
        "max_tokens": 50
    }

    try:
        response = requests.post(URL, headers=headers, json=test_data)
        if response.status_code == 200:
            print("✅ Conexão com a API estabelecida com sucesso!")
            return True
        else:
            print(f"❌ Erro na conexão: Status {response.status_code}")
            print(f"Resposta: {response.text}")
            return False
    except Exception as e:
        print(f"❌ Erro ao conectar à API: {str(e)}")
        return False

# Se executado diretamente (para teste)
if __name__ == "__main__":
    print("Iniciando sistema de atendimento Neoenergia Elektro...")
    print(f"Usando chave de API: {API_KEY[:5]}...{API_KEY[-4:]}")
    print(f"Modelo: {MODEL}")

    # Verificar se a API está funcionando
    if testar_api():
        atendimento_cliente()
    else:
        print("\nO sistema está operando no modo fallback devido a problemas na API.")
        print("As respostas serão limitadas, mas o atendimento continuará funcionando.")
        atendimento_cliente()

Iniciando sistema de atendimento Neoenergia Elektro...
Usando chave de API: gsk_x...Hshg
Modelo: llama3-70b-8192
Testando conexão com a API Groq...
✅ Conexão com a API estabelecida com sucesso!

=== Sistema de Atendimento Neoenergia Elektro ===

[Atendente] Boa tarde! Sou o atendente virtual da Neoenergia Elektro. Como posso ajudar?

[Cliente] Olá, bom dia! eu estou com algum problema com a energia aqui, pois o meu chuveiro não está ligando
Enviando requisição para: https://api.groq.com/openai/v1/chat/completions

[Atendente] Boa tarde! Peço desculpas pelo incoveniente que você está enfrentando com a energia em sua residência. É frustrante quando não podemos contar com os serviços essenciais em nossos lares.

Posso ajudá-lo a identificar o problema. Como você mencionou que o chuveiro não está ligando, gostaria de saber se outros aparelhos em sua casa estão funcionando corretamente. Além disso, já verificou se o disjuntor ou o fusível responsável pelo chuveiro estão funcionando corretam

Código mais sucinto

In [ ]:
import requests
import json
import datetime
import re
import os
from dotenv import load_dotenv

# Carregar variáveis de ambiente
load_dotenv()

# Configurações da API
API_KEY = os.getenv("GROQ_API_KEY", "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")
MODEL = "llama3-70b-8192"
URL = "https://api.groq.com/openai/v1/chat/completions"

# Base de conhecimento simplificada da Neoenergia Elektro
CATEGORIAS_PROBLEMAS = [
    {"nome": "Falta de energia geral", "criticidade": "Médio", "termos": ["sem luz", "energia caiu", "apagão"], "interno": False},
    {"nome": "Falta de fase", "criticidade": "Médio", "termos": ["falta de fase", "uma fase", "duas fases"], "interno": False},
    {"nome": "Tensão baixa", "criticidade": "Baixo", "termos": ["tensão baixa", "luz fraca"], "interno": False},
    {"nome": "Tensão alta", "criticidade": "Alto", "termos": ["tensão alta", "luz forte", "queimando"], "interno": False},
    {"nome": "Fio partido", "criticidade": "Emergência", "termos": ["fio partido", "cabo rompido", "fio caído"], "seguranca": "Mantenha distância do fio!", "interno": False},
    {"nome": "Fogo na rede", "criticidade": "Emergência", "termos": ["fogo na rede", "fio pegando fogo", "faísca"], "seguranca": "Afaste-se do local e chame os bombeiros!", "interno": False},
    {"nome": "Choque elétrico", "criticidade": "Emergência", "termos": ["choque", "eletrocutado", "levando choque"], "seguranca": "Desligue a energia imediatamente!", "interno": False},
    {"nome": "Disjuntor desarmado", "criticidade": "Baixo", "termos": ["disjuntor desarmou", "disjuntor caiu"], "verificacoes": ["Verifique o disjuntor", "Religue o disjuntor"], "interno": True},
    {"nome": "Problema no medidor", "criticidade": "Médio", "termos": ["medidor parado", "relógio de luz não funciona"], "interno": False},
    {"nome": "Religação de energia", "criticidade": "Médio", "termos": ["religar energia", "voltar a luz", "religação"], "interno": False},
    {"nome": "Ligação nova", "criticidade": "Médio", "termos": ["nova ligação", "primeira vez", "instalação nova"], "interno": False}
]

def obter_saudacao():
    """Retorna uma saudação baseada no horário atual."""
    hora_atual = datetime.datetime.now().hour
    if 5 <= hora_atual < 12:
        return "Bom dia"
    elif 12 <= hora_atual < 18:
        return "Boa tarde"
    else:
        return "Boa noite"

def mensagem_fechamento():
    """Retorna a mensagem padrão de fechamento."""
    return "Agradecemos seu contato! Para mais informações: Site neoenergia.com ou Teleatendimento 116"

def analisar_problema_interno(mensagem):
    """Verifica se a mensagem indica um problema interno."""
    mensagem = mensagem.lower()
    for problema in CATEGORIAS_PROBLEMAS:
        if problema.get("interno", False):
            for termo in problema.get("termos", []):
                if termo.lower() in mensagem:
                    return True, problema.get("verificacoes", [])
    return False, []

def identificar_categoria_problema(mensagem):
    """Identifica a categoria de problema com base na mensagem."""
    mensagem = mensagem.lower()
    # Verificar emergências primeiro
    for problema in CATEGORIAS_PROBLEMAS:
        if problema.get("criticidade") == "Emergência":
            for termo in problema.get("termos", []):
                if termo.lower() in mensagem:
                    return problema
    # Verificar outros problemas
    for problema in CATEGORIAS_PROBLEMAS:
        for termo in problema.get("termos", []):
            if termo.lower() in mensagem:
                return problema
    return None

def gerar_protocolo():
    """Gera um número de protocolo baseado na data e hora atuais."""
    agora = datetime.datetime.now()
    return f"{agora.year}{agora.month:02d}{agora.day:02d}{agora.hour:02d}{agora.minute:02d}{agora.second:02d}"

def consultar_ia(mensagem_cliente, contexto_anterior=None):
    """Consulta a IA para obter resposta ao cliente."""
    e_interno, dicas_internas = analisar_problema_interno(mensagem_cliente)
    problema = identificar_categoria_problema(mensagem_cliente)
    protocolo = gerar_protocolo() if not e_interno and problema else None

    sistema_prompt = f"""
    Você é o atendente virtual da Neoenergia Elektro. Seja objetivo e claro.

    Informações:
    - Mensagem: {mensagem_cliente}
    - Problema interno: {"Sim" if e_interno else "Não"}
    {f"- Dicas: {', '.join(dicas_internas)}" if dicas_internas else ""}
    {f"- Problema: {problema['nome']}" if problema else ""}
    {f"- Criticidade: {problema['criticidade']}" if problema else ""}
    {f"- Protocolo: {protocolo}" if protocolo else ""}

    Regras:
    1. Respostas curtas (1-2 frases)
    2. Para problemas internos: dê apenas 1 verificação
    3. Emergências: comece com "ATENÇÃO!" e inclua segurança
    4. Use linguagem simples
    5. Inclua protocolo quando aplicável
    """

    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    messages = [{"role": "system", "content": sistema_prompt}]
    if contexto_anterior:
        messages.extend(contexto_anterior)
    messages.append({"role": "user", "content": mensagem_cliente})

    data = {
        "model": MODEL,
        "messages": messages,
        "temperature": 0.7,
        "max_tokens": 100
    }

    try:
        response = requests.post(URL, headers=headers, json=data)
        return response.json()["choices"][0]["message"]["content"]
    except Exception:
        return criar_resposta_fallback(e_interno, problema, protocolo)

def criar_resposta_fallback(e_interno, problema, protocolo):
    """Resposta de fallback mais curta"""
    if e_interno:
        return "Verifique os disjuntores. Se persistir, nos avise."
    elif problema and problema.get("criticidade") == "Emergência":
        return f"ATENÇÃO! {problema.get('seguranca', 'Mantenha-se afastado!')} Protocolo: {protocolo}"
    elif problema:
        return f"Registrado. Protocolo: {protocolo}. Iremos resolver."
    else:
        return "Poderia detalhar melhor o problema?"

def atendimento_cliente():
    """Função principal para iniciar atendimento."""
    print("\n=== Atendimento Neoenergia Elektro ===\n")
    historico = []
    primeira_mensagem = True

    while True:
        if primeira_mensagem:
            print(f"[Atendente] {obter_saudacao()}! Como posso ajudar?")
            primeira_mensagem = False

        mensagem_cliente = input("\n[Cliente] ").strip()

        if mensagem_cliente.lower() in ["sair", "tchau", "encerrar", "obrigado", "agradeço"]:
            print(f"\n[Atendente] {mensagem_fechamento()}")
            break

        resposta = consultar_ia(mensagem_cliente, historico)
        historico.append({"role": "user", "content": mensagem_cliente})
        historico.append({"role": "assistant", "content": resposta})

        print(f"\n[Atendente] {resposta}")

if __name__ == "__main__":
    atendimento_cliente()


=== Atendimento Neoenergia Elektro ===

[Atendente] Boa tarde! Como posso ajudar?

[Cliente] olá estou sem uma das fases em casa, caiu a banana do poste

[Atendente] Entendi! Você está sem uma das fases em casa e a "banana" do poste caiu. Isso pode ser um problema de falta de energia elétrica. Vou registrar sua ocorrência e enviar uma equipe para verificar e resolver o problema o mais rápido possível. Por favor, aguarde um pouco e faremos o que for necessário para restabelecer a energia em sua casa.

[Cliente] obrigao pela atenção

[Atendente] De nada! Estamos aqui para ajudar. Já registramos sua ocorrência e faremos o possível para resolver o problema o mais rápido possível.

[Cliente] sair

[Atendente] Agradecemos seu contato! Para mais informações: Site neoenergia.com ou Teleatendimento 116
